In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import pandas as pd
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import socks
import socket
import requests
from bs4 import BeautifulSoup

In [9]:
df = pd.read_csv("_PysaScraping.csv")

df

,Company Name,Website,Date of Attack,Industry,Headquarters
0,My Cloud Star,https://mycloudstar.com,08/16/21,NaN,NaN
1,HPW,http://www.hpw.com,04/29/21,NaN,NaN
2,HVAC Facility & Plant Maintenance Tools,https://www.goodway.com/,04/20/21,NaN,NaN
3,Danko Emergency Equipment,https://danko.net/,04/21/21,NaN,NaN
4,FCS Financial,https://www.myfcsfinancial.com/,04/18/21,NaN,NaN
...,...,...,...,...,...
215,Allard,https://www.allard-europe.com/,05/05/20,NaN,NaN
216,Diamond Box,http://diamond-box.co.uk,05/01/20,NaN,NaN
217,Faic,http://faic.it,04/23/20,NaN,NaN
218,Liberty Linehaul,http://www.libertylinehaul.com,04/03/20,NaN,NaN


In [4]:
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)

old_socket = socket.socket # saving old socket info 
socket.socket = socks.socksocket

socket.socket = old_socket # reuse the original socket for selenium

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_linux64.zip
Driver has been saved in cache [/home/kali/.wdm/drivers/chromedriver/linux64/94.0.4606.61]


In [5]:
# glassdoor search
company_list = []
industry_list = []
location_list = []
number = 0

for i in df["Company Name"]: 
    try:
#         driver.get('https://www.google.com')
#         search = driver.find_element(By.NAME, 'q')
#         search.send_keys(i + ' glassdoor overview')
#         search.send_keys(Keys.RETURN)
        driver.get("https://www.google.com/search?q=" + i + " glassdoor overview")
        
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element_by_tag_name('h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        company_list.append(np.nan)
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + i + " | " + industry + " | " + location)
    number = number + 1
    time.sleep(2)
    

0) My Cloud Star | Internet | Jacksonville, FL
1) HPW | Computer Hardware & Software | Palo Alto, CA
2) HVAC Facility & Plant Maintenance Tools | Computer Hardware & Software | Palo Alto, CA
3) Danko Emergency Equipment | Computer Hardware & Software | Palo Alto, CA
4) FCS Financial | Computer Hardware & Software | Jefferson City, MO
5) HABI | Computer Hardware & Software | 
6) Woodlake Unified | Computer Hardware & Software | 
7) Haverhill High School | Preschool & Child Care | Haverhill, MA
8) Spartanburg & Pelham OB-GYN | Preschool & Child Care | Haverhill, MA
9) Grupo SAN | Preschool & Child Care | Don Torcuato, Argentina
10) Jesse Engineering Co | Preschool & Child Care | Tacoma, WA
11) One Community Health | Healthcare Services & Hospital | Sacramento, CA
12) Logansport Community School Corporation | Healthcare Services & Hospital | Logansport, IN
13) Betson Enterprises | Sport & Recreation | Carlstadt, NJ
14) Karavan Trailers | Sport & Recreation | Fox Lake, WI
15) WPSD Local 6 

In [10]:
df['Industry'] = industry_list
df['Headquarters'] = location_list

df

,Company Name,Website,Date of Attack,Industry,Headquarters
0,My Cloud Star,https://mycloudstar.com,08/16/21,Internet,"Jacksonville, FL"
1,HPW,http://www.hpw.com,04/29/21,Computer Hardware & Software,"Palo Alto, CA"
2,HVAC Facility & Plant Maintenance Tools,https://www.goodway.com/,04/20/21,NaN,NaN
3,Danko Emergency Equipment,https://danko.net/,04/21/21,NaN,NaN
4,FCS Financial,https://www.myfcsfinancial.com/,04/18/21,NaN,"Jefferson City, MO"
...,...,...,...,...,...
215,Allard,https://www.allard-europe.com/,05/05/20,NaN,"Central, Hong Kong"
216,Diamond Box,http://diamond-box.co.uk,05/01/20,NaN,"West Bromwich, United Kingdom"
217,Faic,http://faic.it,04/23/20,NaN,
218,Liberty Linehaul,http://www.libertylinehaul.com,04/03/20,NaN,"Montebello, CA"


In [11]:
df.to_csv('PysaWithNA.csv')